In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Import Bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6
from bokeh.models import ColorBar
from bokeh.models import DatetimeTickFormatter
from bokeh.models import SingleIntervalTicker, LinearAxis
from bokeh.models import FixedTicker
from bokeh.models import HoverTool

output_notebook()

def smooth_curve(x, y):
    # Use cubic spline interpolation to smooth the bounary line a tad
    f1 = interp1d(x, y,kind='cubic')
    df_s = pd.DataFrame()
    new_index = np.linspace(x.min(),x.max(),len(y)*100)
    df_s['smooth'] = f1(new_index)
    df_s.index = new_index

    return new_index, df_s['smooth']

def add_histogram_to_chart(p1, x, legend, series_color, n_bins=20, with_bars=False, y_axis=True, bin_list=None):
    hist, edges = np.histogram(x, density=False, bins=n_bins if bin_list is None else bin_list)
    
    # Use the `quad` renderer to display the histogram bars.
    if with_bars:
        #bar_edges = edges - (edges[1]-edges[0])/2
        bar_edges = edges
        p1.quad(top=hist, bottom=0, left=bar_edges[:-1], right=bar_edges[1:], line_color=series_color,line_width=1, alpha=0.6, fill_color=None)

    smooth_ind, smooth_vals = smooth_curve(edges[:-1], hist)
    p1.line(smooth_ind, smooth_vals, line_color=series_color, line_width=5, alpha=0.6, legend=legend, name='line')
    p1.yaxis.visible = y_axis


Loading BokehJS ...

# Question 1

Sample from the exponential distribution using numpy.random.exponential (note that the scale parameter is 1/λ). Plot a histogram of your samples, and calculate the sample mean and variance.

In [2]:
# Create dataframe
df = np.random.exponential(10, 10000)

# create a new figure
p1 = figure(title="Distribution of exponential random values for λ=0.1", plot_width=980, plot_height=300, toolbar_location='right')
# Add the histogram
add_histogram_to_chart(p1, df, None, '#0080ff', 50, True)
show(p1)

# Question 2
1. Derive an analytical solution for the MLE of $\lambda$
2. Plot the error of the MLE as a function of the sample size.

### 1. Analytical solution for:
<h2><center>
$\boldsymbol{\hat\lambda}= \textrm{arg max }\mathcal{L}(\lambda;X)$</center></h2>

The probability density function for $\lambda >0$ is given by:

<h5><center>$f(x) = \lambda\exp^{-\lambda x} \textrm{ when }x \geqslant 0$</h5></center>

Since the terms of the sequence are independent and identically distributed, the likelyhood function is equal to the product of their densities:
<h5><center>$\begin{align}\mathcal{L}(\lambda;x_1,\cdots ,x_n)&=\prod_{j=1}^{n}f(x_j;\lambda) \\ 
&=\prod_{j=1}^{n}\lambda \exp(-\lambda x_j) \\ 
&=\lambda^n\exp\Bigg(-\lambda\sum_{j=1}^{n}x_j\Bigg)\end{align}$</center></h5>

The log-likelyhood function is:
<h5><center>$\begin{align}\ell(\lambda;x_1,\cdots ,x_n)&=\ln(\mathcal{L}(\lambda;x_1,\cdots ,x_n)) \\ 
&=\ln\Bigg(\lambda^n\exp\Bigg(-\lambda\sum_{j=1}^{n}x_j\Bigg)\Bigg) \\ 
&=n\ln\lambda-\lambda\sum_{j=1}^{n}x_j\end{align}$</center></h5>


The derivative of the likelihood estimator is 0 when it reaches a local maximum:
<h5><center>$\begin{align}\frac{d}{d\lambda}\ell(\lambda;x_1,\cdots ,x_n)&=0 \\ \\
\frac{d}{d\lambda}\Bigg(n\ln\lambda-\lambda\sum_{j=1}^{n}x_j\Bigg)&=0 \\
\frac{n}{\lambda}-\sum_{j=1}^{n}x_j&=0\end{align}$</center></h5>

And we can infer:
<h3><center>$\boxed{\hat\lambda=\dfrac{n}{\sum_{j=1}^{n}x_j}}$</center></h3>

In [3]:
# 2. MLE error
def MLE_error(lbda, sample_size):
    df = np.random.exponential(1/lbda, sample_size)
    mle_lambda = sample_size / df.sum()
    
    return 100*abs(lbda - mle_lambda) / lbda

In [4]:
sample_sizes = np.arange(100, 10000, 100)
lbda = 0.1
errors = np.array([MLE_error(lbda, sample_size) for sample_size in sample_sizes])
smooth_ind, smooth_vals = smooth_curve(sample_sizes, errors)

# create a new figure
p2 = figure(title="Maximum Likelyhood Estimation errors for λ=%s" % str(lbda), plot_width=980, plot_height=300, toolbar_location='right')

# turn off minor ticks
p2.xaxis.minor_tick_line_color = None
p2.yaxis.minor_tick_line_color = None

# Add the line
p2.line(smooth_ind, smooth_vals, color='#80ff00', line_color='#80ff00', line_width=5, alpha=0.6)

# Update the axes
p2.xaxis.axis_label = 'Sample Size'
p2.yaxis.axis_label = 'Estimation Error (%)'
show(p2)